In [1]:
import pandas as pd


def trainScaledManhattan(in_vec):
    mean = in_vec.mean()
    std = in_vec.std()
    return mean,std


def classifyScaledManhattan(mean,std, q):

    if(len(mean) != len(q)):
        print (len(mean))
        print(len(q))
        print("Be sure that both vectors are the same dimension!")
        return
    for i in range(len(mean)):
        if (std[i] == 0):
            std[i] = 1e-10

    return sum([abs(mean[i]-q[i])/std[i] for i in range(len(q))])

c:\users\jw\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jw\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\jw\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#from sklearn.preprocessing import StandardScaler
#ss = StandardScaler()

def train (subject,vectors):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:50]  #前50个样本训练，后200个样本测试
    mean,std = trainScaledManhattan(subject_vecs)
    
    scores = []   
    for i in range(len(subject_vecs)):
        score = classifyScaledManhattan(mean,std,subject_vecs.iloc[i])
        #print(possitive_score)
        scores.append(score)
        
    threshold = search_threshold(scores,0.05)
    #subject_vecs = ss.fit_transform(subject_vecs)    
    
    return mean,std,threshold

def test(subject,vectors):
    mean,std,threshold = train(subject,vectors)    #调用训练
    #print('mean:',mean)
    #print('std:',std)
    possitive_samples = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[50:400]  #正样本
    #possitive_samples = ss.fit_transform(possitive_samples)
    negative_samples = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)
    #negative_samples = ss.fit_transform(negative_samples)
    
    #print(possitive_samples)
    #print(negative_samples)
    
    possitive_scores = []
    negative_scores = []
    
    for i in range(len(possitive_samples)):
        possitive_score = classifyScaledManhattan(mean,std,possitive_samples.iloc[i])
        #print(possitive_score)
        possitive_scores.append(possitive_score)
        
    for i in range(len(negative_samples)):
        negative_score = classifyScaledManhattan(mean,std,negative_samples.iloc[i])
        #print(negative_score)
        negative_scores.append(negative_score)
    return possitive_scores,negative_scores,threshold

def search_threshold(scores,threshold):
    scores_asc = sorted(scores)
    #print(scores_asc)
    threshold_index = int(len(scores)*(1-threshold))
    return scores_asc[threshold_index]

In [3]:
file_name = './data/CMU_oral_output.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','sessionIndex','rep'],axis=1)

subject_list = ['s002','s032','s036','s047','s052']
for subject in subject_list:
    positive_scores,negative_scores,threshold = test(subject,vectors)
    negative1 = sum(positive_scores>threshold)
    negative2 = sum(negative_scores<threshold)
    miss_rate1 = negative1/350
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1950
    
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

s002 miss_rate1 is: 0.46
s002 miss_rate2 is: 0.03875
s002 miss_rate is: 0.11435897435897435
s032 miss_rate1 is: 0.38
s032 miss_rate2 is: 0.23375
s032 miss_rate is: 0.26
s036 miss_rate1 is: 0.022857142857142857
s036 miss_rate2 is: 0.0
s036 miss_rate is: 0.0041025641025641026
s047 miss_rate1 is: 0.3342857142857143
s047 miss_rate2 is: 0.138125
s047 miss_rate is: 0.17333333333333334
s052 miss_rate1 is: 0.09142857142857143
s052 miss_rate2 is: 0.0
s052 miss_rate is: 0.01641025641025641


In [4]:
file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)

subject_list = ['s002','s032','s036','s047','s052']
for subject in subject_list:
    positive_scores,negative_scores,threshold = test(subject,vectors)
    negative1 = sum(positive_scores>threshold)
    negative2 = sum(negative_scores<threshold)
    miss_rate1 = negative1/350
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1950
    
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

s002 miss_rate1 is: 0.8228571428571428
s002 miss_rate2 is: 0.359375
s002 miss_rate is: 0.44256410256410256
s032 miss_rate1 is: 0.32
s032 miss_rate2 is: 0.433125
s032 miss_rate is: 0.4128205128205128
s036 miss_rate1 is: 0.3314285714285714
s036 miss_rate2 is: 0.0075
s036 miss_rate is: 0.06564102564102564
s047 miss_rate1 is: 0.38
s047 miss_rate2 is: 0.27625
s047 miss_rate is: 0.2948717948717949
s052 miss_rate1 is: 0.84
s052 miss_rate2 is: 0.275
s052 miss_rate is: 0.3764102564102564


matrix：这就是用户之间的评分或观点等你衡量的东西或你的业务商品的原始矩阵。
row_columns：如果你衡量的是列之间的距离，则设为 1；如果你衡量的是行之间的距离，则设为 0；
size：所得矩阵的所需大小。也就是说，当寻找用户或商品相似度时，这就是用户或商品的数量。
所以如果有 500 个不同用户，则距离矩阵的大小就为 500×500。

In [5]:
from scipy import spatial
import numpy as np


def adjusted_cos_distance_matrix(size, matrix, row_column):
    distances = np.zeros((size,size))
    if row_column == 0:
        M_u = matrix.mean(axis=1)
        #print('M_u is:',M_u)
        m_sub = matrix.as_matrix() - M_u[:,None]
        #print('m_sub is:',m_sub)
    if row_column == 1:
        M_u = matrix.df.as_matrix().T.mean(axis=1)
        m_sub = matrix.T - M_u[:,None]
    for first in range(0,size):
        for sec in range(0,size):
            distance = spatial.distance.cosine(m_sub[first],m_sub[sec])
            distances[first,sec] = distance
    return distances


def adjusted_cos_distance(vector1,vector2):
    M_u1 = vector1.mean()
    M_u2 = vector2.mean()
    m_sub1 = vector1 - M_u1
    m_sub2 = vector2 - M_u2
    distance = spatial.distance.cosine(m_sub1,m_sub2)
    return distance

def cos_distance_matrix(size, matrix, row_column):
    distances = np.zeros((size,size))
    if row_column == 0:
        m_sub = matrix.as_matrix() 
        #print('m_sub is:',m_sub)
    if row_column == 1:
        M_u = matrix.df.as_matrix().T.mean(axis=1)
        m_sub = matrix.T
    for first in range(0,size):
        for sec in range(0,size):
            distance = spatial.distance.cosine(m_sub[first],m_sub[sec])
            distances[first,sec] = distance
    return distances

def cos_distance(vector1,vector2):
    m_sub1 = vector1
    m_sub2 = vector2
    distance = spatial.distance.cosine(m_sub1,m_sub2)
    return distance


#中心点对应的index,0.05false alarm的distance阀值
def center (subject,vectors):    #根据前200个样本计算中心点
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:50]
    distances = adjusted_cos_distance_matrix(50,subject_vecs,0)
    #print(distances)
    distances_sum = pd.DataFrame(distances).sum().tolist()
    #print(distances_sum)
    center_index  = distances_sum.index(min(distances_sum))
    return subject_vecs.iloc[center_index]

def cosine_threshold (center_vec,vectors):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:50]
    possitive_scores = []
    for i in range(len(subject_vecs)):
        distance = adjusted_cos_distance(center_vec,subject_vecs.iloc[i])
        possitive_scores.append(distance)
    #print(sorted(possitive_scores))
    threshold = search_threshold(possitive_scores,0.05)
    return threshold
    

def test_possitive(subject,vectors,center_vec,distance_threshold): 
    positive_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[50:400]
    possitive_scores = []
    for i in range(len(positive_vecs)):
        distance = adjusted_cos_distance(center_vec,positive_vecs.iloc[i])
        possitive_scores.append(distance)
    #print(possitive_scores)
    negative1 = sum(possitive_scores>distance_threshold)
    #print(negative1)
    return negative1

def test_passitive(subject,vectors,center_vec,distance_threshold):
    negative_vecs = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)    
    negative_scores = []
    for i in range(len(negative_vecs)):
        distance = adjusted_cos_distance(center_vec,negative_vecs.iloc[i])
        negative_scores.append(distance)
    #print(sorted(negative_scores))
    negative2 = sum(negative_scores<distance_threshold)
    #print(negative2)
    return negative2  

In [6]:
import pandas as pd

file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)

subject_list = ['s002','s032','s036','s047','s052']
#subject_list = ['s002']
for subject in subject_list:
    center_vec = center(subject,vectors)
    distance_threshold =  cosine_threshold(center_vec,vectors)
    print(distance_threshold)
    negative1 = test_possitive(subject,vectors,center_vec,distance_threshold)
    negative2 = test_passitive(subject,vectors,center_vec,distance_threshold)

    miss_rate1 = negative1/350
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1950
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

c:\users\jw\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


0.5312842682488612
s002 miss_rate1 is: 0.5971428571428572
s002 miss_rate2 is: 0.73125
s002 miss_rate is: 0.7071794871794872
0.5421177578960874
s032 miss_rate1 is: 0.09428571428571429
s032 miss_rate2 is: 0.65625
s032 miss_rate is: 0.5553846153846154
0.006287581285940869
s036 miss_rate1 is: 0.33714285714285713
s036 miss_rate2 is: 0.004375
s036 miss_rate is: 0.0641025641025641
0.4392009093012582
s047 miss_rate1 is: 0.16857142857142857
s047 miss_rate2 is: 0.625625
s047 miss_rate is: 0.5435897435897435
0.13390295426194176
s052 miss_rate1 is: 0.4657142857142857
s052 miss_rate2 is: 0.295625
s052 miss_rate is: 0.3261538461538461


In [7]:
#中心点对应的index,0.05false alarm的distance阀值
def center2 (subject,vectors):    #根据前200个样本计算中心点
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:50]
    distances = cos_distance_matrix(50,subject_vecs,0)
    #print(distances)
    distances_sum = pd.DataFrame(distances).sum().tolist()
    #print(distances_sum)
    center_index  = distances_sum.index(min(distances_sum))
    return subject_vecs.iloc[center_index]

def cosine_threshold2 (center_vec,vectors):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:50]
    possitive_scores = []
    for i in range(len(subject_vecs)):
        distance = cos_distance(center_vec,subject_vecs.iloc[i])
        possitive_scores.append(distance)
    #print(sorted(possitive_scores))
    threshold = search_threshold(possitive_scores,0.05)
    return threshold
    

def test_possitive2(subject,vectors,center_vec,distance_threshold): 
    positive_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[50:400]
    possitive_scores = []
    for i in range(len(positive_vecs)):
        distance = cos_distance(center_vec,positive_vecs.iloc[i])
        possitive_scores.append(distance)
    #print(possitive_scores)
    negative1 = sum(possitive_scores>distance_threshold)
    #print(negative1)
    return negative1

def test_passitive2(subject,vectors,center_vec,distance_threshold):
    negative_vecs = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)    
    negative_scores = []
    for i in range(len(negative_vecs)):
        distance = cos_distance(center_vec,negative_vecs.iloc[i])
        negative_scores.append(distance)
    #print(sorted(negative_scores))
    negative2 = sum(negative_scores<distance_threshold)
    #print(negative2)
    return negative2 

In [8]:
import pandas as pd

file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)

subject_list = ['s002','s032','s036','s047','s052']
#subject_list = ['s002']
for subject in subject_list:
    center_vec = center2(subject,vectors)
    distance_threshold =  cosine_threshold2(center_vec,vectors)
    print(distance_threshold)
    negative1 = test_possitive2(subject,vectors,center_vec,distance_threshold)
    negative2 = test_passitive2(subject,vectors,center_vec,distance_threshold)

    miss_rate1 = negative1/350
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1950
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

c:\users\jw\anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0.3491409699450472
s002 miss_rate1 is: 0.5914285714285714
s002 miss_rate2 is: 0.726875
s002 miss_rate is: 0.7025641025641025
0.3887816563516677
s032 miss_rate1 is: 0.045714285714285714
s032 miss_rate2 is: 0.633125
s032 miss_rate is: 0.5276923076923077
0.006083499452264607
s036 miss_rate1 is: 0.4142857142857143
s036 miss_rate2 is: 0.00375
s036 miss_rate is: 0.07743589743589743
0.28275140555185097
s047 miss_rate1 is: 0.13428571428571429
s047 miss_rate2 is: 0.435625
s047 miss_rate is: 0.38153846153846155
0.11641889077863621
s052 miss_rate1 is: 0.48
s052 miss_rate2 is: 0.293125
s052 miss_rate is: 0.32666666666666666


s002 miss_rate is: 0.385
s032 miss_rate is: 0.63375
s036 miss_rate is: 0.000625
s047 miss_rate is: 0.239375
s052 miss_rate is: 0.493125

s002 miss_rate is: 0.290625
s032 miss_rate is: 0.314375
s036 miss_rate is: 0.0
s047 miss_rate is: 0.475
s052 miss_rate is: 0.0

s002 miss_rate is: 0.5375
s032 miss_rate is: 0.7125
s036 miss_rate is: 0.000625
s047 miss_rate is: 0.27375
s052 miss_rate is: 0.6125